#Scraping: récupération des fichiers pdf du site du professeur Max de Wilde

Dans ce notebook, nous créons un robot qui va scraper la page du professeur et récupérer le titre de tous les fichiers PDF et les stocker dans un fichier. 

Étapes 1 et 2 : OK

#Imports

In [1]:
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

#Récupération de tous les 51 articles de la page d'accueil

Étape 3 : OK

In [2]:


url = "https://max.de.wilde.web.ulb.be/camille"
try:
    response = requests.get(url, timeout=10, verify=False)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    
    print("Page Title:", soup.title.string if soup.title else "Camille")
    print("\nPage Text:")
    print(soup.get_text(strip=True, separator='\n'))
    
    print("\nLinks found:")
    for link in soup.find_all('a'):
        href = link.get('href', 'No href')
        text = link.get_text(strip=True) or 'No text'
        print(f"- {href}: {text}")
except Exception as e:
    print(f"Error: {e}")

Error: HTTPSConnectionPool(host='max.de.wilde.web.ulb.be', port=443): Max retries exceeded with url: /camille (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001F763308F10>, 'Connection to max.de.wilde.web.ulb.be timed out. (connect timeout=10)'))


#Amélioration du scraping et de enregistrement des 51 fichiers PDF dans un dossier sans erreurs de connexion SSL.

In [3]:
import requests
from bs4 import BeautifulSoup
import urllib3
import os
from urllib.parse import urljoin

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

articles = []


root_url = "https://max.de.wilde.web.ulb.be/camille"
folder = "camille_pdfs"
os.makedirs(folder, exist_ok=True)

try:
    response = requests.get(url, timeout=10, verify=False)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    for link in soup.find_all('a', href=True):
        href = link['href']
        if href.lower().endswith('.pdf'):
            pdf_url = urljoin(url, href)
            filename = os.path.join(folder, os.path.basename(href))
            print(f"Downloading: {pdf_url}")
            pdf_response = requests.get(pdf_url, verify=False)
            with open(filename, 'wb') as f:
                f.write(pdf_response.content)
            print(f"Saved: {filename}")
    print(f"\nAll PDFs saved to '{folder}' folder")
except Exception as e:
    print(f"Error: {e}")

Error: HTTPSConnectionPool(host='max.de.wilde.web.ulb.be', port=443): Max retries exceeded with url: /camille (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001F7640BD350>, 'Connection to max.de.wilde.web.ulb.be timed out. (connect timeout=10)'))


 #APIs pour enrichir CAMille

 Étape 4 : OK

 #Imports

In [4]:
import json
import requests
import os
import requests
import PyPDF2
import re


In [5]:


API_TOKEN = "eRtDMbakSqTGKfQXRFOblqoLQSVLwkTf"  # https://www.ncdc.noaa.gov/cdo-web/token
headers = {"token": API_TOKEN}

for file in os.listdir("camille_pdfs"):
    if file.endswith('.pdf'):
        try:
            with open(f"camille_pdfs/{file}", 'rb') as f:
                text = " ".join([p.extract_text() for p in PyPDF2.PdfReader(f).pages])
        except Exception as e:
            print(f"Skipping {file}: {e}")
            continue
        
        dates = re.findall(r'\d{4}-\d{2}-\d{2}', text)[:3]
        locations = list(set(re.findall(r'\b[A-Z][a-z]+\b', text)))[:3]
        
        print(f"\n{file}: Dates={dates}, Locations={locations}")
        
        for date in dates[:1]:
            for loc in locations[:1]:
                r = requests.get(f"https://www.ncdc.noaa.gov/cdo-web/api/v2/data", 
                                headers=headers, params={"locationid": f"CITY:{loc}", 
                                "startdate": date, "enddate": date, "datasetid": "GHCND"})
                print(f"{loc} on {date}: {r.json() if r.ok else 'No data'}")


KB_JB92_1885-09-29_01-00002.pdf: Dates=[], Locations=['Cham', 'Maujan', 'Hundorts']


In [6]:
pip install PyMuPDF


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\aspng\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
import fitz  # PyMuPDF pour lecture des PDF
import os

# Chemin vers le dossier contenant les PDF
pdf_folder = "camille_pdfs"
pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith(".pdf")]

# Sélection du premier fichier PDF
pdf_path = os.path.join(pdf_folder, pdf_files[10])

# Ouverture et lecture du fichier PDF
doc = fitz.open(pdf_path)

# Extraction du texte
title = f"Titre du document : {pdf_files[10]}"
paragraphs = []

for page in doc:
    text = page.get_text()
    paragraphs.append(text.strip())

doc.close()

# Affichage
content = "\n".join(paragraphs)
print(title)
print("==================================")
print(content)


IndexError: list index out of range

#Nettoyage du texte à l'aide d'expressions régulières

In [80]:
# Suppression de tout ce qui se trouve entre parenthèses
clean_content = re.sub("\([^\)]+\)", " ", content)
# Suppression des espaces multiples
clean_content = re.sub("\s+", " ", clean_content)

print(content)
print("==================================")
print(clean_content)



4 Ianvier 193S 
L/V L I B R E 
B E L G I Q U E 
EXTERIEUR 
*»• 
' •* 
ALLEMAGNE 
LES AGRICULTEURS 
DE LA PRUSSE 
ORIENTALE 
DEMANDENT 
PROTECTION 
Kooiiicsbcrg. 3. — .-a chambre *!'*• 
priculture do la l'rus.s. Orientale u adres-
sé au chancelier von Schleicher et au 
miuietro do l'Intérieur 
vou Braun, un 
télé-zrammo demandant oue des mesures 
soient prises immédiatement nour empê-
cher rentrée en AHemanne des beurres, 
graisses, la-rd-s et orotluits analogues su-
perflus 
UNE MOTION 
H I T L E R I E N N E 
CONTRE LES JOCKEYS ETRANGERS 
Berlin, 3. — Le parti hitlérien a dé-
pOùô à la Dièto do Prusse une motion 
demandant que les étrangers ne puissent 
pi-us s'entraîner 
cn Allemagne 
comme 
jockey, é t a n t donné que (es postes les 
plus importants dans le sport 
hippique 
allemand sont occupés par des étrangers, 
nui font passer dans leur pays des gains 
énormes tandis que les jockeys allemands 
meurent de faim. 
ANGLETERRE 
UNE 
CARGAISON 
D'OR SUD-AFRI-
CAIN 
A R R I V E 
A 
LOND

#Création d'un fichier avec le contenu de l'article

In [82]:
with open("../tp1/camille_example.txt", "w") as writer:
    writer.write(f"{title}\n\n{clean_content}")

#Utiliser Free OCR API (comme celle proposée par [OCR.space](https://ocr.space/ocrapi)) permet extraire automatiquement du texte à partir d’images, y compris des fichiers .png.

Étape 5 : OK

In [ ]:
import os
import fitz
import requests

os.makedirs("camille_images", exist_ok=True)
API_KEY = "K83927014588957"  # clé a demander sur OCR.space
for pdf in os.listdir("camille_pdfs"):
    if not pdf.endswith(".pdf"): continue
    doc = fitz.open(f"camille_pdfs/{pdf}")
    for i, page in enumerate(doc):
        for img in page.get_images(full=True):
            xref = img[0]
            pix = fitz.Pixmap(doc, xref)
            if pix.n < 5:
                pix.save(f"camille_images/{pdf}_{i}_{xref}.png")
            else:
                fitz.Pixmap(fitz.csRGB, pix).save(f"camille_images/{pdf}_{i}_{xref}.png")
            with open(f"camille_images/{pdf}_{i}_{xref}.png", "rb") as f:
                r = requests.post("https://api.ocr.space/parse/image",
                                  files={"file": f}, data={"apikey": API_KEY})
            print(pdf, i, "->", r.status_code)

KB_JB230_1892-08-07_01-0003.pdf 0 -> 200
KB_JB230_1903-10-16_01-0002.pdf 0 -> 200
KB_JB230_1913-07-05_01-0001.pdf 0 -> 200
KB_JB258_1884-09-03_01-0003.pdf 0 -> 200
KB_JB258_1894-12-09_01-0003.pdf 0 -> 200
KB_JB258_1906-01-09_01-0002.pdf 0 -> 200
KB_JB421_1899-05-15_01-00003.pdf 0 -> 200
KB_JB421_1926-10-29_01-00002.pdf 0 -> 200
KB_JB421_1950-04-15_01-00004.pdf 0 -> 200
KB_JB427_1920-01-10_01-00004.pdf 0 -> 200
KB_JB427_1933-01-04_01-00003.pdf 0 -> 200
KB_JB427_1949-07-18_01-00008.pdf 0 -> 200
KB_JB449_1846-05-30_01-00002.pdf 0 -> 200
KB_JB449_1912-01-04_01-00003.pdf 0 -> 200
KB_JB449_1947-08-29_01-00003.pdf 0 -> 200
KB_JB494_1853-10-30_01-0002.pdf 0 -> 200
KB_JB494_1922-09-28_01-0005.pdf 0 -> 200
KB_JB494_1939-12-08_01-0004.pdf 0 -> 200
KB_JB555_1836-02-08_01-00002.pdf 0 -> 200
KB_JB555_1899-01-19_01-00003.pdf 0 -> 200
KB_JB555_1940-03-01_01-00004.pdf 0 -> 200
KB_JB567_1857-02-02_01-00003.pdf 0 -> 200
KB_JB567_1892-01-03_01-00005.pdf 0 -> 200
KB_JB567_1924-08-30_01-00003.pdf 0 -> 200
K

Analyse du resultat : ces fichiers .png sont les articles de presse, car les fichiers PDF sont en réalité des images des articles scannés.

#Utiliser NCDC CDO web services API ( [NCEI CDO](https://www.ncei.noaa.gov/)) pour d’extraire automatiquement données de la meteo.

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

station_id = "GHCND:BE000065417"  # Bruxelles-Uccle
base_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data" 
api_token = "eRtDMbakSqTGKfQXRFOblqoLQSVLwkTf" #Clé a demander sur https://www.ncdc.noaa.gov/cdo-web/token

params = {
    'datasetid': 'GHCND',
    'stationid': station_id,
    'startdate': '2023-01-01',
    'enddate': '2024-01-01',
    'limit': 1000
}

headers = {'token': api_token}
resp = requests.get(base_url, headers=headers, params=params)
data = resp.json()

try:
    results = data['results']
    print(f"Données météo pour {station_id}:")
    for record in results:
        print(f"- {record['date']}: {record['value']} {record['datatype']}")
except KeyError:
    print("Aucune donnée disponible ou erreur de station")

Aucune donnée disponible ou erreur de station
